# 🤖 AI 기반 머신러닝 서울 맛집 추천 시스템
이 프로젝트는 sample 리뷰 데이터를 불러와 전처리하고,
지하철역, 음식종류, 예산, 연령층 등 점수를 계산하여 추천하는 과정을 담습니다.

In [ ]:
import gradio as gr
import pandas as pd
import numpy as np
import gradio as gr
import lightgbm as lgb
from sklearn.model_selection import train_test_split
# ------------------------
# 1. 데이터 로드 및 모델 학습
df = pd.read_excel('/content/drive/MyDrive/Restaurants.xlsx')
# ------------------------
# CSV 불러오기
df = pd.read_csv('../data/sample_reviews.csv')
df.head()